# Qwen3-VL-8B - Complete Jupyter Notebook Guide

**Ziel**: Qwen3-VL-8B Modell herunterladen, laden und verwenden

**Anforderungen**:
- GPU mit mindestens 16GB VRAM (besser 24GB+)
- Hugging Face Account (optional, aber empfohlen)
- Internet-Verbindung für Download (~16GB)

**Zeit**: ~30 Minuten für Download + Setup

## Schritt 1: Pakete installieren

In [ ]:
# Installieren Sie die notwendigen Pakete
!pip install --upgrade transformers torch torchvision torchaudio
!pip install accelerate bitsandbytes peft pillow
!pip install huggingface-hub

## Schritt 2: Hugging Face Login (Optional aber empfohlen)

In [ ]:
# Hugging Face Token eingeben
# Gehen Sie zu https://huggingface.co/settings/tokens und erstellen Sie einen Token

from huggingface_hub import login

# Option 1: Interaktiv (wird Sie auffordern, Token einzugeben)
login()

# Option 2: Mit Token direkt
# login(token="hf_xxxxxxxxxxxxx")

## Schritt 3: GPU-Status prüfen

In [ ]:
import torch
import sys

print("="*60)
print("SYSTEM INFORMATION")
print("="*60)

print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ CUDA not available! Model loading will be slow.")

print("="*60)

## Schritt 4: Modell und Tokenizer herunterladen

⚠️ **Warnung**: Dies wird ~16GB herunterladen und dauert 10-30 Minuten je nach Internetgeschwindigkeit

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-VL-8B"

print(f"Downloading {model_name}...")
print("This may take 10-30 minutes depending on your internet speed.")
print()

# Tokenizer herunterladen
print("Step 1: Downloading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
print("✓ Tokenizer downloaded successfully")
print(f"  Vocab size: {len(tokenizer)}")
print()

# Modell herunterladen
print("Step 2: Downloading model (this is the large part)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Speichern als float16 um Speicher zu sparen
    device_map="auto",  # Automatisch auf GPU laden
    trust_remote_code=True
)
print("✓ Model downloaded successfully")
print(f"  Parameters: {model.num_parameters() / 1e9:.2f}B")
print(f"  Model size: {sum(p.numel() * p.element_size() for p in model.parameters()) / 1e9:.2f} GB")
print()

print("="*60)
print("✓ MODEL AND TOKENIZER READY!")
print("="*60)

## Schritt 5: Speicher-Optimierung (Falls nötig)

In [ ]:
# Falls Sie Speicherprobleme haben, verwenden Sie Quantization
# Dies reduziert Speicher um ~75% mit minimalem Qualitätsverlust

from transformers import BitsAndBytesConfig
import torch

# 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Modell mit Quantization laden (optional)
# model = AutoModelForCausalLM.from_pretrained(
#     "Qwen/Qwen3-VL-8B",
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True
# )

print("Quantization config prepared (optional)")
print("Uncomment the code above if you need to save memory")

## Schritt 6: Einfacher Text-Inference

In [ ]:
# Einfache Text-Generierung

prompt = "What is machine learning?"

# Text tokenisieren
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generieren
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

# Dekodieren
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("="*60)
print("PROMPT:")
print(prompt)
print()
print("RESPONSE:")
print(response)
print("="*60)

## Schritt 7: Vision-Fähigkeiten testen (mit Bild)

In [ ]:
# Vision-Fähigkeiten: Bild beschreiben
from PIL import Image
import requests

# Beispiel: Bild von URL laden
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

try:
    image = Image.open(requests.get(image_url, stream=True).raw)
    
    # Prompt mit Bild
    prompt = "Describe this image in detail."
    
    # Für Vision-Modelle brauchen wir spezielle Verarbeitung
    # (Dies ist ein Beispiel - die genaue Implementierung hängt vom Modell ab)
    
    print("Image loaded successfully")
    print(f"Image size: {image.size}")
    print()
    print("Note: Vision processing requires model-specific preprocessing.")
    print("Check Qwen3-VL documentation for exact vision API.")
    
except Exception as e:
    print(f"Error loading image: {e}")
    print("You can use local images instead:")
    print("  image = Image.open('/path/to/your/image.jpg')")

## Schritt 8: Batch-Inferenz (mehrere Prompts)

In [ ]:
# Mehrere Prompts auf einmal verarbeiten

prompts = [
    "What is Python?",
    "Explain neural networks",
    "What is deep learning?"
]

print("Processing batch of prompts...")
print()

for i, prompt in enumerate(prompts, 1):
    print(f"[{i}/{len(prompts)}] {prompt}")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=150,
            temperature=0.7,
            do_sample=True
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Response: {response[:200]}...")
    print()

## Schritt 9: Speicher-Monitoring

In [ ]:
import torch
import psutil

print("="*60)
print("MEMORY USAGE")
print("="*60)

# GPU Memory
if torch.cuda.is_available():
    print("GPU Memory:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1e9
        reserved = torch.cuda.memory_reserved(i) / 1e9
        total = torch.cuda.get_device_properties(i).total_memory / 1e9
        
        print(f"  GPU {i}:")
        print(f"    Allocated: {allocated:.2f} GB")
        print(f"    Reserved: {reserved:.2f} GB")
        print(f"    Total: {total:.2f} GB")
        print(f"    Free: {total - allocated:.2f} GB")
else:
    print("GPU Memory: Not available")

# CPU Memory
print()
print("CPU Memory:")
memory = psutil.virtual_memory()
print(f"  Used: {memory.used / 1e9:.2f} GB")
print(f"  Available: {memory.available / 1e9:.2f} GB")
print(f"  Total: {memory.total / 1e9:.2f} GB")
print(f"  Percent: {memory.percent}%")

print("="*60)

## Schritt 10: Modell speichern (lokal)

In [ ]:
# Modell lokal speichern um es später schneller zu laden

save_path = "./qwen3-vl-8b-local"

print(f"Saving model to {save_path}...")

# Modell speichern
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✓ Model saved successfully")
print()
print("Later, you can load it faster with:")
print(f"  model = AutoModelForCausalLM.from_pretrained('{save_path}')")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{save_path}')")

## Schritt 11: Modell aus lokaler Kopie laden (schneller)

In [ ]:
# Falls Sie das Modell später wieder laden wollen (schneller als Download)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

save_path = "./qwen3-vl-8b-local"

print(f"Loading model from {save_path}...")

# Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(save_path, trust_remote_code=True)

# Modell laden
model = AutoModelForCausalLM.from_pretrained(
    save_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("✓ Model loaded successfully from local copy")
print(f"  Parameters: {model.num_parameters() / 1e9:.2f}B")

## Schritt 12: Cleanup (Speicher freigeben)

In [ ]:
# Falls Sie mit dem Modell fertig sind und Speicher freigeben möchten

import torch
import gc

print("Cleaning up...")

# Modell und Tokenizer löschen
del model
del tokenizer

# Garbage collection
gc.collect()

# GPU Cache leeren
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✓ Memory cleaned up")

# Speicher prüfen
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    print(f"GPU memory allocated: {allocated:.2f} GB")

## Tipps & Tricks

### Download-Optionen:
```python
# Option 1: Automatisch auf GPU laden (empfohlen)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-VL-8B",
    device_map="auto"
)

# Option 2: Auf CPU laden (langsamer, aber weniger VRAM)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-VL-8B",
    device_map="cpu"
)

# Option 3: Mit Quantization (speichersparend)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-VL-8B",
    quantization_config=bnb_config,
    device_map="auto"
)
```

### Speicher-Anforderungen:
- **Full Precision (float32)**: ~32GB VRAM
- **Half Precision (float16)**: ~16GB VRAM
- **4-bit Quantization**: ~4GB VRAM

### Download-Speicherort:
```bash
# Standard: ~/.cache/huggingface/hub/
# Ändern mit:
export HF_HOME=/custom/path
```

### Schnellerer Download:
```bash
# Mit git-lfs (falls installiert)
git clone https://huggingface.co/Qwen/Qwen3-VL-8B
```

## Troubleshooting

### Problem: "CUDA out of memory"
**Lösung**: 
- Verwenden Sie float16 statt float32
- Verwenden Sie 4-bit Quantization
- Reduzieren Sie batch_size

### Problem: "Model not found"
**Lösung**:
- Login zu Hugging Face: `huggingface-cli login`
- Prüfen Sie Internetverbindung
- Versuchen Sie mit `trust_remote_code=True`

### Problem: "Slow download"
**Lösung**:
- Speichern Sie das Modell lokal nach dem Download
- Verwenden Sie einen besseren Internet-Connection
- Versuchen Sie zu einer anderen Zeit

### Problem: "Kernel crashes"
**Lösung**:
- Starten Sie den Kernel neu
- Reduzieren Sie max_length in generate()
- Verwenden Sie Quantization